In [3]:
#Winning Factor 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# ML
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

print('Import Done')


Import Done


In [19]:
drivers = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\DRIVERS.csv", sep=';')
results = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\RESULTS.csv", sep=';')
races = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\RACES.csv", sep=',')
constructors = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\CONSTRUCTORS.csv", sep=',')
qualifying = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\QUALIFYING.csv", sep=',')
status = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\STATUS.csv", sep=',')
pit_stops = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\PIT_STOPS.csv", sep=',')
lap_times = pd.read_csv(r"C:\Development\workspace\SideQuests\Repo\McLaren_Performance_Analysis_Team14\Raw Data\LAP_TIMES.csv", sep=',')


print('Load Done')

Load Done


In [22]:

print("drivers:", drivers.columns.tolist())
print("results:", results.columns.tolist())
print("races:", races.columns.tolist())
print("constructors:", constructors.columns.tolist())
print("qualifying:", qualifying.columns.tolist())
print("status:", status.columns.tolist())
print("pit_stops:", pit_stops.columns.tolist())
print("lap_times:", lap_times.columns.tolist())

# Normalize all column names to uppercase
results.columns = results.columns.str.upper()
races.columns = races.columns.str.upper()
drivers.columns = drivers.columns.str.upper()
constructors.columns = constructors.columns.str.upper()
status.columns = status.columns.str.upper()
qualifying.columns = qualifying.columns.str.upper()

# Merge Tables
merged = results.merge(races, on='RACEID', suffixes=('', '_RACE')) \
                .merge(drivers, on='DRIVERID') \
                .merge(constructors, on='CONSTRUCTORID') \
                .merge(status, on='STATUSID', how='left') \
                .merge(qualifying[['RACEID', 'DRIVERID', 'Q1', 'Q2', 'Q3']], on=['RACEID', 'DRIVERID'], how='left')

results 


drivers: ['DRIVERID', 'DRIVERREF', 'NUMBER', 'CODE', 'FORENAME', 'SURNAME', 'DOB', 'NATIONALITY', 'URL']
results: ['RESULTID', 'RACEID', 'DRIVERID', 'CONSTRUCTORID', 'NUMBER', 'GRID', 'POSITION', 'POSITIONTEXT', 'POSITIONORDER', 'POINTS', 'LAPS', 'TIME', 'MILLISECONDS', 'FASTESTLAP', 'RANK', 'FASTESTLAPTIME', 'FASTESTLAPSPEED', 'STATUSID']
races: ['RACEID', 'YEAR', 'ROUND', 'CIRCUITID', 'NAME', 'DATE', 'TIME', 'URL', 'FP1_DATE', 'FP1_TIME', 'FP2_DATE', 'FP2_TIME', 'FP3_DATE', 'FP3_TIME', 'QUALI_DATE', 'QUALI_TIME', 'SPRINT_DATE', 'SPRINT_TIME']
constructors: ['CONSTRUCTORID', 'CONSTRUCTORREF', 'NAME', 'NATIONALITY', 'URL;']
qualifying: ['QUALIFYID', 'RACEID', 'DRIVERID', 'CONSTRUCTORID', 'NUMBER', 'POSITION', 'Q1', 'Q2', 'Q3']
status: ['STATUSID', 'STATUS']
pit_stops: ['RACEID', 'DRIVERID', 'STOP', 'LAP', 'TIME', 'DURATION', 'MILLISECONDS']
lap_times: ['RACEID', 'DRIVERID', 'LAP', 'POSITION', 'TIME', 'MILLISECONDS']


,RESULTID,RACEID,DRIVERID,CONSTRUCTORID,NUMBER,GRID,POSITION,POSITIONTEXT,POSITIONORDER,POINTS,LAPS,TIME,MILLISECONDS,FASTESTLAP,RANK,FASTESTLAPTIME,FASTESTLAPSPEED,STATUSID
0,1,18,1,1,22.0,1,1.0,1,1,10.0,58,1:34:50.616,5690616.0,39.0,2.0,1:27.452,218.300,1
1,2,18,2,2,3.0,5,2.0,2,2,8.0,58,+5.478,5696094.0,41.0,3.0,1:27.739,217.586,1
2,3,18,3,3,7.0,7,3.0,3,3,6.0,58,+8.163,5698779.0,41.0,5.0,1:28.090,216.719,1
3,4,18,4,4,5.0,11,4.0,4,4,5.0,58,+17.181,5707797.0,58.0,7.0,1:28.603,215.464,1
4,5,18,5,1,23.0,3,5.0,5,5,4.0,58,+18.014,5708630.0,43.0,1.0,1:27.418,218.385,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26754,26760,1144,825,210,20.0,14,16.0,16,16,0.0,57,NaN,NaN,57.0,1.0,1:25.637,222.002,11
26755,26761,1144,859,215,30.0,12,17.0,17,17,0.0,55,NaN,NaN,52.0,12.0,1:28.751,214.212,5
26756,26762,1144,822,15,77.0,9,NaN,R,18,0.0,30,NaN,NaN,14.0,19.0,1:29.482,212.462,130
26757,26763,1144,861,3,43.0,20,NaN,R,19,0.0,26,NaN,NaN,5.0,17.0,1:29.411,212.631,5


In [21]:
#Mercedes
mercedes = merged[merged['name'] == 'Mercedes'].copy()


KeyError: 'name'